In [6]:
CONFIG = {
    "DATA_FOLDER": "survey_results",
    "FILES": ["survey_2022.csv", "survey_2023.csv"],
    "YEARS": [2022, 2023],
    "SPLIT_BY": "business unit",
    "OUTPUT_FILE_COMPARISON": "nps_comparison.csv",
    "OUTPUT_FILE_TOTAL": "nps_total.csv"
}



In [8]:
import pandas as pd
import os

def load_and_prepare_data(file_name, year, config):
    df = pd.read_csv(os.path.join(config["DATA_FOLDER"], file_name), encoding='utf-8')
    df['externalId'] = df['externalId'].str.split('_').str[0]
    return df.rename(columns={'Score': f'Score {year}', 'business unit': f'{config["SPLIT_BY"]}_{year}'})

# Load and prepare data
dfs = {year: load_and_prepare_data(file, year, CONFIG) for year, file in zip(CONFIG["YEARS"], CONFIG["FILES"])}

# Merge data
df_common = pd.merge(dfs[2022], dfs[2023], how='inner', on='externalId')

def calculate_nps(df, year, split_by):
    nps = df.groupby(f'{split_by}_{year}').apply(
        lambda x: ((x[f'Score {year}'] >= 9).sum() - (x[f'Score {year}'] < 7).sum()) / len(x) * 100
    ).reset_index()
    nps.columns = [split_by, f'NPS score {year}']
    return nps

def calculate_respondents(df, year, split_by):
    respondents = df.groupby(f'{split_by}_{year}').count().reset_index()
    respondents = respondents[[f'{split_by}_{year}', f'Score {year}']]
    respondents.columns = [split_by, f'Common Respondents {year}']
    return respondents

# Calculate NPS and respondent counts
nps_scores = [calculate_nps(df_common, year, CONFIG["SPLIT_BY"]) for year in CONFIG["YEARS"]]
respondents = [calculate_respondents(df_common, year, CONFIG["SPLIT_BY"]) for year in CONFIG["YEARS"]]

# Merge all data
nps_comparison = pd.merge(
    pd.merge(nps_scores[0], nps_scores[1], on=CONFIG["SPLIT_BY"]),
    pd.merge(respondents[0], respondents[1], on=CONFIG["SPLIT_BY"]),
    on=CONFIG["SPLIT_BY"]
)

# Calculate the change in NPS scores
nps_comparison['change'] = nps_comparison['NPS score 2023'] - nps_comparison['NPS score 2022']

# Save the result to a CSV file
nps_comparison.to_csv(CONFIG["OUTPUT_FILE_COMPARISON"], index=False)

# Function to calculate NPS components
def nps_components(x, year):
    promoters = (x[f'Score {year}'] >= 9).sum()
    passives = ((x[f'Score {year}'] >= 7) & (x[f'Score {year}'] < 9)).sum()
    detractors = (x[f'Score {year}'] < 7).sum()
    total = promoters + passives + detractors
    return pd.Series([promoters, passives, detractors, promoters - detractors, promoters / total, passives / total, detractors / total], 
                     index=[f'Promoters {year}', f'Passives {year}', f'Detractors {year}', f'NPS {year}', f'Promoters share {year}', f'Passives share {year}', f'Detractors share {year}'])

# Calculate the NPS scores and components
nps_total = pd.concat([nps_components(df_common, year) for year in CONFIG["YEARS"]])

# Save the result to a CSV file
nps_total.to_csv(CONFIG["OUTPUT_FILE_TOTAL"], index=True)

In [53]:
import pandas as pd

# Read the CSV files
df_2022 = pd.read_csv('survey_results/survey_2022.csv', encoding='utf-8')
df_2023 = pd.read_csv('survey_results/survey_2023.csv', encoding='utf-8')

df_2022['externalId'] = df_2022['externalId'].str.split('_').str[0]
df_2023['externalId'] = df_2023['externalId'].str.split('_').str[0]

# Filter the common respondents based on 'externalId'
df_common = pd.merge(df_2022, df_2023, how='inner', on='externalId', suffixes=('_2022', '_2023'))

# Rename the columns for clarity
df_common.rename(columns={'Score_2022': 'Score 2022', 'Score_2023': 'Score 2023'}, inplace=True)

# Calculate the NPS scores for 2022
nps_2022 = df_common.groupby('division_2022').apply(lambda x: ((x['Score 2022'] >= 9).sum() - (x['Score 2022'] < 7).sum()) / len(x) * 100).reset_index()
nps_2022.columns = ['division', 'NPS score 2022']

# Calculate the respondents count for 2022
respondents_2022 = df_common.groupby('division_2022').count().reset_index()
respondents_2022 = respondents_2022[['division_2022', 'Score 2022']]
respondents_2022.columns = ['division', 'Common Respondents 2022']

# Repeat the process for the 2023 data
nps_2023 = df_common.groupby('division_2023').apply(lambda x: ((x['Score 2023'] >= 9).sum() - (x['Score 2023'] < 7).sum()) / len(x) * 100).reset_index()
nps_2023.columns = ['division', 'NPS score 2023']

# Calculate the respondents count for 2023
respondents_2023 = df_common.groupby('division_2023').count().reset_index()
respondents_2023 = respondents_2023[['division_2023', 'Score 2023']]
respondents_2023.columns = ['division', 'Common Respondents 2023']

# Merge the NPS scores for 2022 and 2023, as well as the respondent counts
nps_comparison = pd.merge(pd.merge(nps_2022, nps_2023, on='division'), pd.merge(respondents_2022, respondents_2023, on='division'), on='division')

# Calculate the change in NPS scores
nps_comparison['change'] = nps_comparison['NPS score 2023'] - nps_comparison['NPS score 2022']

# Save the result to a CSV file
nps_comparison.to_csv('nps_comparison_division.csv', index=False)


In [64]:
import pandas as pd

# Specify 'business unit' or 'division'
SPLIT_BY = 'division'

def nps(score):
    """Calculate NPS category based on score."""
    if score >= 9:
        return 'Promoter'
    elif score >= 7:
        return 'Passive'
    else:
        return 'Detractor'

def calculate_nps(df):
    """Group by business unit and calculate NPS and respondents count."""
    df['Category'] = df['Score'].apply(nps)
    result = df.groupby(SPLIT_BY).apply(lambda x: pd.Series({
        'NPS': ((x['Category'] == 'Promoter').sum() - (x['Category'] == 'Detractor').sum()) / len(x) * 100,
        'Respondents': len(x)
    }))
    return result

def format_results(result, year):
    """Format the results with NPS score and respondent count."""
    result[f'{year} NPS'] = result['NPS'].round(1).astype(str) + ' (' + result['Respondents'].astype(str) + ')'
    return result.drop(columns=['NPS', 'Respondents'])

def read_and_process(year):
    """Read CSV and process data."""
    df = pd.read_csv(f'survey_results/survey_{year}.csv', encoding='utf-8')
    result = calculate_nps(df)
    return format_results(result, year)

def main():
    result_2022 = read_and_process(2022)
    result_2023 = read_and_process(2023)

    # Merge the results
    result = pd.merge(result_2022, result_2023, on=SPLIT_BY, how='outer').fillna(0)

    # Calculate and format the changes
    result['Change'] = (result['2023 NPS'].str.extract(r"(\d+.\d+)").astype(float).fillna(0) - result['2022 NPS'].str.extract(r"(\d+.\d+)").astype(float).fillna(0)).round(1).astype(str) + \
                       ' (' + (result['2023 NPS'].str.extract(r"\((\d+)\)").fillna(0).astype(int) - result['2022 NPS'].str.extract(r"\((\d+)\)").fillna(0).astype(int)).astype(str) + ')'

    # Save to CSV
    result.to_csv('nps_comparison_total_'+SPLIT_BY.replace(' ', '_')+'.csv', index=True)

if __name__ == "__main__":
    main()
